<a href="https://colab.research.google.com/github/Ayushverma135/Whisper-Hindi-ASR-model-IIT-Bombay-Intership/blob/main/ASR_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Check GPU

In [ ]:
!nvidia-smi

Sat Apr 20 18:18:33 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Installing whisper

In [ ]:
! pip install git+https://github.com/openai/whisper.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.4 MB/s eta 0:00:00


# Load and Import Library and Model

In [ ]:
import whisper
model = whisper.load_model("large") #769M Parameter

100%|█████████████████████████████████████| 2.88G/2.88G [00:57<00:00, 54.1MiB/s]


# Display the Audio

In [ ]:
from IPython.display import Audio, display

display(Audio('01-04397-02.mp3', autoplay=True))

# Transcribing into text

In [ ]:
result = model.transcribe("01-04397-02.mp3",language='hi')
print(result["text"])

 देमानी और भ्रताचारी की वज़न से जर्ण पतिनी भी अधिकारी वो विचोईय माला माल हो रहे हैं।


In [ ]:
!pip install SpeechRecognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 36.2 MB/s eta 0:00:00


In [ ]:
!pip install ffmpeg-python

In [ ]:
!pip install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 18.4 MB/s eta 0:00:00


# Load and Display all the Audios from the drive

In [ ]:
import os
import speech_recognition as sr
import ffmpeg
from jiwer import wer
# Function to recognize speech from an audio file
def recognize_speech(audio_file):
    # recognizer = sr.Recognizer()
    # with sr.AudioFile(audio_file) as source:
    #     audio_data = recognizer.record(source)
    wav_audio_file = audio_file.split('.')[0] + '.wav'
    ffmpeg.input(audio_file).output(wav_audio_file).run(overwrite_output=True)

    # Read the WAV audio file
    with open(wav_audio_file, 'rb') as f:
        audio_data = f.read()
    # with open(audio_file, 'rb') as f:
    #     audio_data = f.read()
    # result = model.transcribe("audio_file",language='hi')
    # return result["text"]
    try:
        text = recognizer.recognize_google(audio_data, language='hi-IN')
        return text
    except sr.UnknownValueError:
        return "Google Speech Recognition could not understand the audio"
    except sr.RequestError as e:
        return "Could not request results from Google Speech Recognition service; {0}".format(e)

# Path to the folder containing audio files
folder_path = '/content/drive/MyDrive/Audio'

# Open a text file to write the output
output_file_path = '/content/drive/MyDrive/ResultText3.txt'
# with open(output_file_path, 'w') as output_file:
#     # Loop through all files in the folder
#     for filename in os.listdir(folder_path):
#         if filename.endswith('.mp3'):  # Check if the file is an MP3 file
#             file_path = os.path.join(folder_path, filename)
#             # Convert MP3 to WAV format using ffmpeg
#             wav_file_path = os.path.splitext(file_path)[0] + '.wav'
#             ffmpeg.input(file_path).output(wav_file_path).run(overwrite_output=True)
#             # Recognize speech from the audio file

#             recognized_text = recognize_speech(wav_file_path)

#             # Write the recognized text to the output file
#             output_file.write(f"File: {filename}\n")
#             output_file.write(f"Recognized Text: {recognized_text}\n\n")
#             # Remove the temporary WAV file
#             os.remove(wav_file_path)

with open(output_file_path, 'w') as output_file:
    # Initialize variables for WER calculation
    total_words = 0
    total_errors = 0
    i=0
    # Loop through all files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.mp3'):  # Check if the file is an MP3 file
            file_path = os.path.join(folder_path, filename)
            # Convert MP3 to WAV format using ffmpeg
            # wav_file_path = os.path.splitext(file_path)[0] + '.wav'
            # ffmpeg.input(file_path).output(wav_file_path).run(overwrite_output=True)
            # Recognize speech from the audio file
            # recognized_text = recognize_speech(wav_file_path)
            recognized_text = recognize_speech(file_path)
            # Load ground truth transcription
            ground_truth_file = os.path.splitext(file_path)[0] + '.txt'
            with open(ground_truth_file, 'r', encoding='utf-8') as f:
                ground_truth_text = f.read().strip()
            # Calculate WER
            current_wer = wer(ground_truth_text, recognized_text)
            # Write the recognized text, ground truth, and WER to the output file
            output_file.write(f"File: {filename}\n")
            output_file.write(f"Recognized Text: {recognized_text}\n")
            output_file.write(f"Ground Truth: {ground_truth_text}\n")
            output_file.write(f"Word Error Rate: {current_wer}\n\n")
            # Update WER variables
            total_words += len(ground_truth_text.split())
            total_errors += current_wer * len(ground_truth_text.split())
            # Remove the temporary WAV file
            # os.remove(wav_file_path)
            i+=1
            if(i==100):
              break

    # Calculate overall WER
    overall_wer = total_errors / total_words
    # Write overall WER to the output file
    output_file.write(f"Overall Word Error Rate: {overall_wer}\n")

RuntimeError: Failed to load audio: ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --ena